In [1]:
import pandas as pd
import json
import Tool
import os
import config
import numpy  as np
import operators_v4
import tqdm
from joblib import Parallel, delayed
import sys
sys.path.append("C:/Users/User/我的雲端硬碟 (owen.lin@mutual-boost.com)/MBQ_Tej_v2/MBQ_tej_v2")
import MBQ_tej_v2_manager
import matplotlib.pyplot as plt
args = json.load(open("./config.json", encoding="utf-8"))
save_path = r'C:\Users\User\Desktop\pc\stcok\Results\Results.xlsx'

In [2]:
Handler_cache_path = args['Handler_cache_path']
print(f"Load Data : {Handler_cache_path}")
CacheHandler = Tool.Handler(path = Handler_cache_path,data_type = args['data_type'])

Load Data : C:\Users\User\Documents\MBQ_TW_DB


In [3]:
# import 更新因子资料
handler = MBQ_tej_v2_manager.Handler()

In [4]:
df_result = pd.read_excel(save_path,index_col=0)
import fitness
df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
df_result['len'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,True))
原始大小 = len(df_result)

use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
use_factor = use_data_df.groupby('value')['key'].apply(list)
use_factor_count = use_factor.str.len().sort_values(ascending=False)
df_result_list = []
top_n = 100
for factor_name in tqdm.tqdm(use_factor_count.index[:]):
    df_result_ = df_result.loc[df_result['cal'].str.contains(factor_name)].sort_values('fitness',ascending=False)
    Top = df_result_.index[:top_n]
    Down = df_result_.index[top_n:]
    if len(Down):
        df_result = df_result.drop(index = Down)
    df_result_list.append(df_result.loc[Top])
best_df_result = pd.concat(df_result_list).drop_duplicates('cal').sort_values('fitness',ascending=False).iloc[:1000]
print(f'因子数{len(best_df_result)}/{原始大小}')
print(f'{best_df_result.fitness.mean()}')

100%|██████████| 14/14 [00:00<00:00, 910.07it/s]

因子数15/15
330.6774053169944


In [5]:
target = eval(args['TARGET_Expr'],CacheHandler)
exp_ret = target = target[np.isfinite(target)]

In [6]:
expr_list = df_result.sort_values('fitness',ascending=False)['cal'].to_list()
index = exp_ret.stack().index
def get_factor_stack(expr,index,args):
    CacheHandler = Tool.Handler(path = args['Handler_cache_path'],data_type = args['data_type'])
    return eval(expr,CacheHandler).stack(dropna = False).reindex(index = index)
factor_df = pd.DataFrame(np.column_stack(Parallel(n_jobs=-1)(delayed(get_factor_stack)(expr,index,args) for expr in tqdm.tqdm(expr_list))),index=index,columns=expr_list)

100%|██████████| 15/15 [00:00<00:00, 1461.09it/s]


In [7]:
def calc_IC_series(factor: pd.DataFrame, exp_ret: pd.DataFrame) -> pd.Series:
    IC_series = factor.corrwith(exp_ret, axis=1, method='spearman')
    return IC_series.sort_index()

def run_IC_test(IC_series: pd.Series, insample_start_date: str, insample_end_date: str) -> pd.DataFrame:
    insample_start = pd.to_datetime(insample_start_date)
    insample_end = pd.to_datetime(insample_end_date)
    outsample_start = pd.to_datetime("2020-01-01")

    IC_mean = pd.Series({
        f"樣本內({insample_start.date()}~{insample_end.date()})": IC_series.loc[insample_start:insample_end].mean(),
        f"樣本外({outsample_start.date()}~{IC_series.index[-1].date()})": IC_series.loc[outsample_start:].mean(),
    })

    ICIR = pd.Series({
        f"樣本內({insample_start.date()}~{insample_end.date()})": IC_mean[0] / IC_series.loc[insample_start:insample_end].std(),
        f"樣本外({outsample_start.date()}~{IC_series.index[-1].date()})": IC_mean[1] / IC_series.loc[outsample_start:].std(),
    })

    return pd.concat({"IC": IC_mean, "ICIR": ICIR}, axis=1)

In [8]:
# 因子檢定模式 = True
# if 因子檢定模式:
#     IC_Se = calc_IC_series(factor_df
#                            ['非负化处理(abs(truediv(signedpower(非负化处理(cs_scale(abs(truediv(signedpower(非负化处理(cs_scale(daily_EMA20, 10)), 30), Volume)), 30)), 3), FAC)))'].unstack(), exp_ret)
#     display(run_IC_test(IC_Se, insample_start_date='2015-01-01', insample_end_date='2019-12-31'))
#     Tool.backtest_factor(factor_df
#                            ['非负化处理(abs(truediv(signedpower(非负化处理(cs_scale(abs(truediv(signedpower(非负化处理(cs_scale(daily_EMA20, 10)), 30), Volume)), 30)), 3), FAC)))'].unstack(), exp_ret, rank_range_n=10, start_date='2019-12-31')
#     IC_Se.ewm(span=60).mean().dropna().iloc[60:].iplot(hline=[IC_Se.mean()])

In [9]:
factor_df.to_pickle('factor_df.pkl')

# 因子整体等权监控

In [10]:
import os
from datetime import time
from datetime import datetime
import numpy as np
import pandas as pd
import json
import pathlib
import tqdm
import Tool
import fitness
def 剔除高度相关(df_result,ts_result,Corr_threshold:float = 0.95):
    要删除的清单 = list()
    #根據因子本身序列評價單因子績效並排序
    #cv_df = ts_result.mean() / ts_result.std()
    #ts_result = ts_result[cv_df.sort_values(ascending = False).index]

    Corr_df = ts_result.corr().abs()
    for factor_expr in ts_result.columns:
        if factor_expr not in 要删除的清单:
            Corr_ = Corr_df.loc[factor_expr:,factor_expr].iloc[1:]
            要删除的清单+=Corr_[Corr_>Corr_threshold].index.to_list()
    要删除的清单 = list(set(要删除的清单))
    if 要删除的清单:
        print(f'发现:{len(要删除的清单)}个表达式与其他其他表达式相关超过{Corr_threshold}，执行删除！！！')
    else:
        print(f'没有相关超过{Corr_threshold}的表达式！！！')
    df_result = df_result.loc[~df_result['cal'].isin(要删除的清单)]
    ts_result = ts_result[df_result['cal'].to_list()]
    return df_result,ts_result

start_time = datetime.now()
print(f"Init variables:{start_time}.")
args = json.load(open("./config.json", encoding="utf-8"))
if args['OUTPUT_DATA_PATH'][:2] == './':
    args['OUTPUT_DATA_PATH'] = str(pathlib.Path().absolute())+args['OUTPUT_DATA_PATH'][1:]
# Data preprocessing
Handler_cache_path = args['Handler_cache_path']
print(f"Load Data : {Handler_cache_path}")
CacheHandler = Tool.Handler(path = Handler_cache_path,data_type = args['data_type'])

Con = eval(args['Con'],CacheHandler)
target = eval(args['TARGET_Expr'],CacheHandler)
target = target[np.isfinite(target) & Con]
print(f"Target : ({args['TARGET_Expr']})[{args['Con']}]")

使用太多次需剔除的data_name = []
if os.path.exists(args['OUTPUT_DATA_PATH']):
    ts_result_path = os.path.join(os.path.dirname(args['OUTPUT_DATA_PATH']),f"{args['OUTPUT_DATA_PATH'].split('/')[-1].replace('.xlsx','')}.parquet")
    ts_result = pd.read_parquet(ts_result_path)
    df_result = pd.read_excel(args['OUTPUT_DATA_PATH'],index_col=0)
    df_result,ts_result = 剔除高度相关(df_result,ts_result,args['Corr_threshold'])
    # 开始确认源资料使用频繁程度
    df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
    df_result = df_result.loc[df_result['fitness']>args['FIT_THRES']]
    print(f"{args['OUTPUT_DATA_PATH']} 已存在，载入历史{len(df_result)}个历史因子")
    use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
    use_factor = use_data_df.groupby('value')['key'].apply(list)
    use_factor_count = use_factor.str.len().sort_values(ascending=False)
    使用太多次需剔除的data_name = use_factor_count.loc[use_factor_count>=args['use_data_count_threshold']].index.to_list()
    if 使用太多次需剔除的data_name:
        print(f'剔除:{使用太多次需剔除的data_name}')
    print(f'Mega Factor Score Up to:{ts_result.mean(axis = 1).mean() / ts_result.mean(axis = 1).std() * 252**0.5}')
else:
    df_result = pd.DataFrame(columns=['cal', 'fitness', 'use_data'])
    ts_result = pd.DataFrame()
count = 0
if args['AMT']=="None" or args['AMT']=="inf": args['AMT'] = np.inf


def factor_to_weight(alpha_array,only_long:bool = False):
    demeaned = alpha_array - np.nanmean(alpha_array,axis = 1)[:, None]
    weights = demeaned / np.nansum(np.abs(demeaned),axis = 1)[:, None]
    weights[np.isnan(weights)] = 0#检查数据无值时视为权重0
    if only_long:
        weights[weights<0] = 0
        weights*=2
    return weights

Full_time_ts_result = pd.concat({expr:factor_to_weight(eval(expr,CacheHandler)[target.notna()],True).mul(CacheHandler['Adj_close'].shift(-2) / CacheHandler['Adj_close'].shift(-1) -1).sum(axis = 1) for expr in tqdm.tqdm(df_result['cal'][:])},axis=1)

Init variables:2025-04-26 14:34:35.409142.
Load Data : C:\Users\User\Documents\MBQ_TW_DB
Target : (Adj_close.shift(-21) / Adj_close.shift(-1) - 1)[(Value_Dollars.rolling(5).mean() > 0)]
没有相关超过0.8的表达式！！！
c:\Users\User\Stock-Futures-Portfolio/Results/Results.xlsx 已存在，载入历史15个历史因子
Mega Factor Score Up to:11.239586854105067


100%|██████████| 15/15 [00:25<00:00,  1.71s/it]


In [11]:
# top15_expr_list = df_result['cal'].head(15).tolist()
# print("前15個因子如下：")
# print(top15_expr_list)

# # 對前 15 個因子進行回測計算
# Full_time_ts_result = pd.concat({
#     expr: factor_to_weight(eval(expr, CacheHandler)[target.notna()], only_long=True)
#     .mul(CacheHandler['Adj_close'].shift(-2) / CacheHandler['Adj_close'].shift(-1) - 1)
#     .sum(axis=1)
#     for expr in tqdm.tqdm(top15_expr_list)
# }, axis=1)

In [12]:
bt_ret = Full_time_ts_result.mean(axis=1)
Tool.show_stats(pd.concat({
    '樣本內':bt_ret.loc[args['DATE_FROM']:args['DATE_TO']],
    '樣本外':bt_ret.loc[args['DATE_TO']:],
    '全樣本':bt_ret.loc[:],
},axis = 1))
bt_ret.cumsum().iplot(vline = [args['DATE_FROM'],args['DATE_TO']])

,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),样本胜率(%),周胜率(%),月胜率(%),年胜率(%),盈亏比(avg_win/avg_loss),总赚赔比(profit_factor),预期报酬(bps),样本数
樣本內,110.14,8.24,8.48,-12.98,-2.61,70.18,81.66,99.07,100.0,1.68,3.98,43.99,2223
樣本外,33.59,5.18,2.40,-14.02,-6.31,62.61,71.21,88.16,100.0,1.44,2.50,17.13,1530
全樣本,180.73,7.01,12.89,-14.02,-2.61,67.09,77.35,94.57,100.0,1.60,3.33,61.20,3753


In [13]:

combined_list = [
    'PBR_TWSE','Total_Operating_Income','Earnings_Before_Interest_and_Tax','Total_Liabilities','Cash_and_Cash_Equivalent','Total_Operating_Income','Basic_Earnings_Per_Share','Return_Rate_on_Equity_A_percent',
    'Total_Assets','Sales_Monthly','Cash_Flow_from_Operating_Activities','Cash_Dividend_Yield_TEJ','Return_on_Total_Assets_A_percent','Net_Operating_Income_Loss'
]


In [14]:
# first = pd.read_parquet('./Results/v4.parquet')
# second = pd.read_parquet('./Results/Results.parquet')
# final = pd.concat([first,second],axis = 1)
# final.to_parquet('./Results/Results.parquet')

In [15]:
manage = MBQ_tej_v2_manager.Handler()

In [16]:
# def manage_factors(CacheHandler):
#     for factor_name in combined_list:
#         if factor_name in CacheHandler.cash_list():
#             continue

#         # 讀入資料並 pivot
#         factor_data = manage(factor_name)
#         if 'mdate' in factor_data.index.names and len(factor_data.columns) > 1:
#             all_factor = factor_data
#         else:
#             Q_columns = [col for col in factor_data.columns if col.endswith('_Q')]
#             required_columns = ['mdate', 'coid'] + Q_columns
#             factor_data_Q = factor_data[required_columns]
#             all_factor = factor_data_Q.pivot(index='mdate', columns='coid', values=Q_columns)
#             if isinstance(all_factor.columns, pd.MultiIndex):
#                 all_factor.columns = all_factor.columns.droplevel(0)

#         all_factor_filtered = all_factor
#         CacheHandler[factor_name] = all_factor_filtered

#     return CacheHandler
# manage_factors(CacheHandler)

In [17]:
def manage_factors(CacheHandler):
    factor_dict = {}

    def add_factor(factor_name, formula):
        factor_dict[factor_name] = formula
        CacheHandler[factor_name] = formula

    add_factor('52_high', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].rolling(252).max())
    add_factor('EP', CacheHandler['Basic_Earnings_Per_Share'] / CacheHandler['Adj_Close'])
    EV = CacheHandler['Value_Dollars'] + CacheHandler['Total_Liabilities'] - CacheHandler['Cash_and_Cash_Equivalent']
    add_factor('EBIT_to_EV', CacheHandler['Earnings_Before_Interest_and_Tax'] / EV)
    bp_quantile = CacheHandler['PBR_TWSE'].rank(pct=True).apply(lambda x: 1 / x)
    roe_quantile = CacheHandler['Return_Rate_on_Equity_A_percent'].rank(pct=True).apply(lambda x: 1 / x)
    add_factor('BPROE_Quantile', bp_quantile + roe_quantile)
    add_factor('Log PS', CacheHandler['Total_Operating_Income'] / CacheHandler['Value_Dollars'].apply(np.log))
    add_factor('One-month change', (CacheHandler['Adj_Close'] - CacheHandler['Adj_Close'].shift(20)) / CacheHandler['Adj_Close'].shift(20))
    add_factor('MoM5', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(5) - 1)
    add_factor('MoM20', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(20) - 1)
    add_factor('MoM60', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(60) - 1)
    add_factor('MoM252', CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift(252) - 1)
    add_factor('Turn20', CacheHandler['Turnover'].rolling(20).mean())
    add_factor('PctTurn', CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1)
    add_factor('PctTurn20', (CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1).rolling(20).mean())
    add_factor('振幅', (CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('振幅換手率因子', ((CacheHandler['Turnover'] / CacheHandler['Turnover'].rolling(40).mean().shift(20) - 1) * ((CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))).rolling(20).mean())
    add_factor('STR', CacheHandler['Turnover'].rolling(20).std())
    add_factor('Vol20', (CacheHandler['Adj_Close'] / CacheHandler['Adj_Close'].shift() - 1).rolling(20).std())
    add_factor('SCR', CacheHandler['Turnover'].rolling(20).std() / CacheHandler['Turnover'].rolling(40, min_periods=1).std().shift(20) - 1)
    add_factor('DRET', (CacheHandler['Adj_Close'] - CacheHandler['Adj_Open']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('FAC', (CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('PLUS', (2 * CacheHandler['Adj_Close'] - CacheHandler['Adj_High'] - CacheHandler['Adj_Low']) / CacheHandler['Adj_Close'].shift(1))
    add_factor('daily_turnover_change', CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1)
    add_factor('daily_turnover_change_ma20', (CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1).rolling(20).mean())
    add_factor('gtr', (CacheHandler['Turnover'] / CacheHandler['Turnover'].shift(1) - 1).rolling(20).std())
    intraday_ret = (CacheHandler['Adj_Close'] - CacheHandler['Adj_Open']) / CacheHandler['Adj_Open']
    add_factor('intraday_return_ewm_vol20', intraday_ret.pct_change().ewm(span=20).mean())

    return factor_dict

factor_dict = manage_factors(CacheHandler)


In [18]:
factor_cols = [col for col in factor_df.columns if col not in ['mdate', 'coid']]
factor_df.reset_index(inplace=True)
factor_dict_gene = {
    factor: factor_df.pivot(index='mdate', columns='coid', values=factor)
    for factor in factor_cols
}

In [19]:
final_factor_dict = {**factor_dict, **factor_dict_gene}

In [20]:
import bottleneck as bn

In [21]:
class FactorCombination:
    def __init__(self, stock_return=None):
        """
        Initialize the FactorCombination class.

        Parameters:
        - stock_return: DataFrame, optional
            Stock return data, typically in decimal format.
        """
        self.stock_return = stock_return

    @staticmethod
    def get_stock_return_close_to_close(daily_return_data):
        """
        Calculate close-to-close stock return from daily percentage return data.

        Parameters:
        - daily_return_data: DataFrame
            Daily percentage return data.

        Returns:
        - DataFrame: Stock returns as decimal values.
        """
        return daily_return_data.div(100)

    @staticmethod
    def get_demean_weighting(data):
        """
        Calculate demeaned and normalized weighting.

        Parameters:
        - data: DataFrame
            Input factor data.

        Returns:
        - DataFrame: Demeaned and normalized weighting.
        """
        df = data.dropna(axis=0, how='all').dropna(axis=1, how='all').copy()
        demean = df.sub(df.mean(axis=1, skipna=True), axis=0)
        weighting = demean.div(demean.abs().sum(axis=1, min_count=1, skipna=True), axis=0)
        weighting = weighting.replace([np.inf, -np.inf], np.nan)
        return weighting

    def combine_factor(self, factor_dict, method='equal_weight', period=250, std_power=1, half_life=126):
        """
        Combine multiple factors using the specified method.

        Parameters:
        - factor_dict: dict
            Dictionary of factors to combine.
        - method: str
            Combination method ('equal_weight', 'double_momentum', 'IC_IR_model', or 'PCA').
        - period: int
            Rolling period for momentum calculation (used in 'double_momentum').
        - std_power: int
            Exponent for standard deviation in momentum weighting (used in 'double_momentum').

        Returns:
        - DataFrame: Combined factor.
        """
        if not factor_dict:
            raise ValueError("factor_dict is empty. Ensure that factors are properly calculated before combining.")

        if method == 'equal_weight':
            multi_factor_weighting = self.get_demean_weighting(list(factor_dict.values())[0]).fillna(0)
            for factor in list(factor_dict.values())[1:]:
                demean_factor = self.get_demean_weighting(factor).fillna(0)
                multi_factor_weighting = multi_factor_weighting.add(demean_factor, fill_value=0)
            factor_all = multi_factor_weighting

        elif method == 'double_momentum':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'double_momentum' method.")
            factor_all = self.double_momentum_model(factor_dict=factor_dict, period=period, std_power=std_power)
        elif method == 'IC_IR_model':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'IC_IR' method.")
            factor_all = self.IC_IR_model(factor_dict=factor_dict, stock_return=self.stock_return, period=period)
        elif method == 'PCA':
            factor_all = self.pca_model(factor_dict=factor_dict)
        elif method == 'half_ic_ir_model':
            if self.stock_return is None:
                raise ValueError("stock_return is required for 'half_ic_ir_model' method.")
            factor_all = self.half_ic_ir_model(factor_dict=factor_dict, period=period, half_life=half_life)   
        elif method == 'XGBoost':
            factor_all = self.xgboost_model(factor_dict=factor_dict)
        else:
            raise ValueError(f"Unknown method '{method}'. Please choose 'equal_weight', 'double_momentum', 'IC_IR_model', or 'half_ic_ir_model'.")

        return factor_all

    def double_momentum_model(self, factor_dict, period=252, std_power=1):
        """
        Double Momentum Model for combining multiple factors without auto direction adjustment.

        Parameters:
        - factor_dict: dict
            Dictionary of factors to combine.
        - period: int
            Rolling period for momentum calculation.
        - std_power: int
            Exponent for standard deviation in momentum weighting.

        Returns:
        - DataFrame: Combined factor with momentum-based weighting.
        """
        print("Step 1: Calculating demean weights...")
        demean_weight_dict = {k: self.get_demean_weighting(v).fillna(0) for k, v in factor_dict.items()}
        for k, v in demean_weight_dict.items():
            print(f"Demeaned weights for factor {k}:\n{v.head()}")

        print("\nStep 2: Calculating long-short returns...")
        ret_ls_dict = {
            k: (w.shift(2) * self.stock_return).sum(axis=1, min_count=1)
            for k, w in demean_weight_dict.items()
        }
        for k, v in ret_ls_dict.items():
            print(f"Long-short returns for factor {k}:\n{v.head()}")

        print("\nStep 3: Preparing df_ret_ls...")
        df_ret_ls = pd.DataFrame(ret_ls_dict).dropna(how='all')
        print("df_ret_ls:\n", df_ret_ls.head())

        print("\nStep 4: Calculating rolling mean and score...")
        rolling_mean_ret = pd.DataFrame(
            bn.move_mean(df_ret_ls.values, window=period, axis=0, min_count=1),
            index=df_ret_ls.index, columns=df_ret_ls.columns
        )
        rolling_std_ret = pd.DataFrame(
            bn.move_std(df_ret_ls.values, window=period, axis=0, min_count=1),
            index=df_ret_ls.index, columns=df_ret_ls.columns
        )
        rolling_score = rolling_mean_ret / (rolling_std_ret ** std_power + 1e-10)
        print("Rolling mean returns:\n", rolling_mean_ret.head())
        print("Rolling score:\n", rolling_score.head())

        print("\nStep 5: Calculating normalized weights...")
        score_weight = (rolling_score.abs().T / rolling_score.abs().sum(axis=1, min_count=1)).T.fillna(0)
        direction = np.sign(rolling_mean_ret).fillna(0)
        factor_weight = score_weight * direction
        factor_weight = factor_weight.clip(lower=0) * 2  # long-only + boost
        print("Factor weight (adjusted):\n", factor_weight.head())

        print("\nStep 6: Combining factors...")
        combined_factor = pd.DataFrame(0, index=self.stock_return.index, columns=self.stock_return.columns)
        for k in demean_weight_dict:
            w = factor_weight[k].reindex(demean_weight_dict[k].index)
            df = demean_weight_dict[k]
            df = df.loc[w.index.intersection(df.index)]
            combined = df.mul(w, axis=0)
            combined_factor = combined_factor.add(combined, fill_value=0)

        print("Final merged factor:\n", combined_factor.head())
        return combined_factor

In [22]:
fc = FactorCombination(stock_return=CacheHandler['Adj_close']/ CacheHandler['Adj_close'].shift() -1)
ic_ir = fc.combine_factor(factor_dict=final_factor_dict, method='double_momentum', period=20)

Step 1: Calculating demean weights...
Demeaned weights for factor 52_high:
coid            1101      1102      1103      1104      1107      1108  \
mdate                                                                    
2011-01-03  0.001036  0.001135  0.001179  0.000455  0.000167  0.000567   
2011-01-04  0.000980  0.001206  0.001083  0.000570  0.000508  0.000550   
2011-01-05  0.000907  0.001200  0.001062  0.000587  0.000971  0.000540   
2011-01-06  0.000876  0.001148  0.001067  0.000572  0.001247  0.000496   
2011-01-07  0.000899  0.001188  0.000891  0.000459  0.001342  0.000433   

coid            1109      1110      1201      1203  ...      9944      9945  \
mdate                                               ...                       
2011-01-03  0.001152  0.001212  0.000628  0.000969  ...  0.001071  0.000488   
2011-01-04  0.001176  0.001137  0.000490  0.000886  ...  0.001194  0.000547   
2011-01-05  0.001124  0.000973  0.000444  0.000853  ...  0.001243  0.000498   
2011-01-06 

In [214]:
factor_df = pd.concat({data_name:final_factor_dict[data_name].stack(future_stack=True) for data_name in tqdm.tqdm(list(final_factor_dict.keys()))},axis=1)
# factor_df.to_pickle('./Data/factor_df.pkl')
# # factor_df = pd.read_pickle('./factor_df.pkl')
factor_df

100%|██████████| 40/40 [00:52<00:00,  1.31s/it]


OSError: Cannot save file into a non-existent directory: 'Data'

In [218]:
from catboost import CatBoostRanker, Pool

def split_large_groups(X, y, group_id, max_group_size=1023):
    new_X, new_y, new_group_id = [], [], []
    current_group = 0
    for g in np.unique(group_id):
        mask = (group_id == g)
        X_g = X[mask]
        y_g = y[mask]
        n = len(X_g)
        if n <= max_group_size:
            new_X.append(X_g)
            new_y.append(y_g)
            new_group_id += [current_group] * n
            current_group += 1
        else:
            for i in range(0, n, max_group_size):
                X_chunk = X_g.iloc[i:i + max_group_size]
                y_chunk = y_g.iloc[i:i + max_group_size]
                new_X.append(X_chunk)
                new_y.append(y_chunk)
                new_group_id += [current_group] * len(X_chunk)
                current_group += 1

    X_new = pd.concat(new_X)
    y_new = pd.concat(new_y)
    group_id_new = pd.Series(new_group_id, index=y_new.index)
    return X_new, y_new, group_id_new


重新訓練模型 = True

if 重新訓練模型:
    print('準備 CatBoost 所需數據')
    time_slice = '2018-12-31'
    start_date = exp_ret.index[0].date()
    end_date = time_slice

    target = (exp_ret.rank(axis=1, pct=True) * 100).round().stack().loc[f'{start_date}':f'{end_date}'].astype(int)
    X = factor_df.loc[target.index].dropna(how='all')
    y = target.loc[X.index]
    group = y.index.get_level_values(0).to_series().astype(str)
    group_id = group.astype('category').cat.codes.values

    # ➤ 拆 group，避免 GPU 超過上限
    X_new, y_new, group_id_new = split_large_groups(X, y, group_id)

    print('開始訓練 CatBoostRanker...')
    model = CatBoostRanker(
        task_type="GPU",
        verbose=True,
        iterations=5000,
        depth=8,
        learning_rate=0.06,
        l2_leaf_reg=10,
        random_strength=3,
    )

    model.fit(Pool(data=X_new, label=y_new, group_id=group_id_new), early_stopping_rounds=50)
    print('模型已訓練完成並儲存')
    model.save_model('cat_model')

準備 CatBoost 所需數據
開始訓練 CatBoostRanker...


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	total: 213ms	remaining: 17m 42s
1:	total: 302ms	remaining: 12m 35s
2:	total: 396ms	remaining: 10m 59s
3:	total: 489ms	remaining: 10m 10s
4:	total: 584ms	remaining: 9m 43s
5:	total: 675ms	remaining: 9m 21s
6:	total: 766ms	remaining: 9m 6s
7:	total: 859ms	remaining: 8m 55s
8:	total: 951ms	remaining: 8m 47s
9:	total: 1.04s	remaining: 8m 41s
10:	total: 1.14s	remaining: 8m 36s
11:	total: 1.23s	remaining: 8m 32s
12:	total: 1.33s	remaining: 8m 29s
13:	total: 1.42s	remaining: 8m 26s
14:	total: 1.52s	remaining: 8m 23s
15:	total: 1.61s	remaining: 8m 22s
16:	total: 1.71s	remaining: 8m 20s
17:	total: 1.8s	remaining: 8m 19s
18:	total: 1.9s	remaining: 8m 18s
19:	total: 1.99s	remaining: 8m 15s
20:	total: 2.08s	remaining: 8m 14s
21:	total: 2.18s	remaining: 8m 13s
22:	total: 2.28s	remaining: 8m 12s
23:	total: 2.37s	remaining: 8m 11s
24:	total: 2.46s	remaining: 8m 10s
25:	total: 2.56s	remaining: 8m 9s
26:	total: 2.65s	remaining: 8m 8s
27:	total: 2.75s	remaining: 8m 8s
28:	total: 2.84s	remaining: 8m 6

In [220]:
from catboost import CatBoostRanker, Pool
# 從文件加載模型
loaded_ranker = CatBoostRanker()
cat_model_path = 'cat_model'
loaded_ranker.load_model(cat_model_path)
print("模型已加載。")
# 使用加載的模型進行預測
y_pred_loaded = loaded_ranker.predict(factor_df)
print("加載模型的預測完成。")
factor = pd.Series(y_pred_loaded,index =  factor_df.index).unstack()

模型已加載。
加載模型的預測完成。


In [176]:
futures = pd.read_csv('20250426011140.csv', encoding='utf-16', sep='\t')
futures['年月日'] = pd.to_datetime(futures['年月日'].astype(str))
futures = futures.sort_values(['標的證券', '證券代碼', '年月日'])

futures['contract_month'] = futures['證券代碼'].str.extract(r'(\d{6})')[0]
futures['contract_month'] = pd.to_datetime(futures['contract_month'], format='%Y%m', errors='coerce')
futures = futures.dropna(subset=['contract_month'])
futures['coid'] = futures['標的證券'].astype(str).str.extract(r'([0-9]{4})')

# === 計算轉倉日（每月第三個週三的前一天）===
def get_third_wednesday_dates(start_date, end_date):
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS')
    third_wednesdays = []
    for first_day in all_months:
        wednesdays = pd.date_range(first_day, first_day + pd.offsets.MonthEnd(0), freq='W-WED')
        if len(wednesdays) >= 3:
            third_wednesdays.append(wednesdays[2])
    return pd.to_datetime(third_wednesdays)

trade_dates = futures['年月日'].drop_duplicates().sort_values()
third_wed = get_third_wednesday_dates(trade_dates.min(), trade_dates.max())
roll_dates = pd.to_datetime([w - pd.Timedelta(days=1) for w in third_wed])
roll_dates_set = set(roll_dates)
roll_dict = {(d.year, d.month): d for d in roll_dates}

def assign_target_contract_month(date):
    year, month = date.year, date.month
    if date in roll_dates_set:
        return pd.Timestamp(f"{year+1}-01-01") if month == 12 else pd.Timestamp(f"{year}-{month+1:02d}-01")
    rd = roll_dict.get((year, month))
    if rd and date > rd:
        return pd.Timestamp(f"{year+1}-01-01") if month == 12 else pd.Timestamp(f"{year}-{month+1:02d}-01")
    return pd.Timestamp(f"{year}-{month:02d}-01")

futures['target_contract_month'] = futures['年月日'].apply(assign_target_contract_month)

# === 選出合約 ===
valid = futures[futures['contract_month'] == futures['target_contract_month']]
idx = valid.groupby(['年月日', 'coid'])['成交張數(量)'].idxmax()
top_contracts = valid.loc[idx].sort_values(['年月日', 'coid']).reset_index(drop=True)

adj_factor = CacheHandler['Adjust_Factor']  
adj_factor.index.name = 'mdate'  
adj_factor = adj_factor.reset_index().melt(id_vars='mdate', var_name='coid', value_name='adjust_factor')
adj_factor = adj_factor.rename(columns={'mdate': 'date'})
adj_factor['date'] = pd.to_datetime(adj_factor['date'])

top_contracts['coid'] = top_contracts['標的證券'].str.extract(r'(\d{4})')
top_contracts['date'] = top_contracts['年月日']
top_contracts = top_contracts.merge(adj_factor, on=['date', 'coid'], how='left')

top_contracts['調整係數'] = top_contracts['adjust_factor'].fillna(1.0)
top_contracts['復權收盤價'] = top_contracts['收盤價'] * top_contracts['調整係數']
top_contracts['復權最低價'] = top_contracts['最低價'] * top_contracts['調整係數']
df = top_contracts.copy()
df = df[['date', 'coid', '報酬率', '成交張數(量)', '復權收盤價','復權最低價']].dropna(subset=['復權收盤價'])
price_matrix = df.pivot(index='date', columns='coid', values='復權收盤價').reindex(columns=ic_ir.columns)
low_matrix = df.pivot(index='date', columns='coid', values='復權最低價').reindex(columns=ic_ir.columns)
exp_ret_fu = price_matrix.shift(-2) / price_matrix.shift(-1) - 1

availability_matrix = (
    df.assign(value=1)
    .pivot_table(index='date', columns='coid', values='value', fill_value=0)
    .astype(bool)
    .reindex(columns=ic_ir.columns, fill_value=0)
)

volume_matrix = df.pivot(index='date', columns='coid', values='成交張數(量)').fillna(0).reindex(columns=ic_ir.columns, fill_value=0)
con_mask = volume_matrix.rolling(20, min_periods=5).mean() > 200
mask = availability_matrix & con_mask

### check

In [24]:
# def check_roll_contract(coid, signal_date):
#     signal_date = pd.to_datetime(signal_date)
#     trade_dates = top_contracts['年月日'].sort_values().unique()

#     # 定義進出場日
#     try:
#         signal_idx = list(trade_dates).index(signal_date)
#         entry_date = trade_dates[signal_idx + 1]
#         exit_date = trade_dates[signal_idx + 2]
#     except IndexError:
#         print("交易日不足，無法計算 t+1, t+2")
#         return

#     for d in [signal_date, entry_date, exit_date]:
#         row = top_contracts[
#             (top_contracts['年月日'] == d) &
#             (top_contracts['標的證券'].astype(str).str.contains(coid))
#         ]
#         if row.empty:
#             print(f"{d.date()} 無對應合約")
#         else:
#             code = row.iloc[0]['證券代碼']
#             cm = row.iloc[0]['contract_month'].strftime("%Y-%m")
#             print(f"{d.date()} 使用合約：{code}，contract_month: {cm}")


In [25]:
# check_roll_contract("世紀鋼", "2025-04-17")  # 假設這是轉倉週的週二

In [26]:
# example_stock = '9958'
# check_df = top_contracts[top_contracts['標的證券'].astype(str).str.contains(example_stock)]
# check_df = check_df[['年月日', '證券代碼', 'contract_month', 'target_contract_month']].sort_values('年月日')
# check_df[:60]

In [221]:
def max_drawdown(prices):
    # 計算累計的最大值
    cumulative_max = prices.cummax()
    # 計算回撤 (Drawdown)
    drawdown = (prices - cumulative_max) / cumulative_max
    # 計算最大回撤 (MDD)
    mdd = drawdown.min()
    return mdd
def show_bt(bt_ret):
    display(pd.concat({"CAGR(%)":bt_ret.mean() * 252,
            'Sharpe' : bt_ret.mean() / bt_ret.std()*252**0.5,
            'Calmar':bt_ret.calmar(),
            'MDD(%)' : bt_ret.max_drawdown()*100,
            '單利MDD(%)' : max_drawdown(bt_ret.cumsum().add(1))*100,
            '樣本勝率(%)' : bt_ret.apply(lambda X:((X.dropna()>0).sum() / X.dropna().shape[0])*100),
            '周勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('W').prod().sub(1)>0).sum() / X.dropna().add(1).resample('W').prod().sub(1).dropna().shape[0])*100),
            '月勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('ME').prod().sub(1)>0).sum() / X.dropna().add(1).resample('ME').prod().sub(1).shape[0])*100),
            '年勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('YE').prod().sub(1)>0).sum() / X.dropna().add(1).resample('YE').prod().sub(1).shape[0])*100),
            'avg_win/avg_loss' : bt_ret.apply(lambda X:(X[X > 0].mean() / abs(X[X < 0].mean()))),
            'profit_factor' : bt_ret.profit_factor(),
            },axis = 1).round(2))
def backtest_factor(factor:pd.DataFrame,exp_ret:pd.DataFrame,rank_range_n:int = 10,start_date:str = '2015-01-01'):
    factor_rank = factor.rank(axis = 1,pct = True,method = 'first')

    IC_Se = factor.corrwith(exp_ret,axis=1,method='spearman').sort_index().loc[start_date:]
    print(f'IC_mean:{round(IC_Se.mean(),4)}')
    print(f'IC_IR:{round(IC_Se.mean()/IC_Se.std(),4)}')

    bt = pd.concat({f'{int(((_/rank_range_n)*100))}% ~ {int((_+1)/rank_range_n*100)}%':exp_ret[(factor_rank>_/rank_range_n) & (factor_rank<=(_+1)/rank_range_n)].mean(axis = 1) - exp_ret.mean(axis=1) for _ in tqdm.tqdm(range(rank_range_n))}, axis = 1).dropna(how = 'all')
    bt = bt.loc[start_date:]
    if (bt.iloc[:,-1] - bt.iloc[:,0]).add(1).prod() > 1:
        bt['LS_ret'] = bt.iloc[:,-1] - bt.iloc[:,0]
    else:
        bt['LS_ret'] = bt.iloc[:,0] - bt.iloc[:,-1]
    show_bt(bt)

    (bt.drop(columns='LS_ret').loc[start_date:].cagr()*100).iplot(kind = 'bar')
backtest_factor(factor,CacheHandler['Adj_Close'].shift(-2) / CacheHandler['Adj_Close'].shift(-1) -1 ,rank_range_n = 10,start_date = '2020-01-01')

IC_mean:0.0628
IC_IR:0.4326


100%|██████████| 10/10 [00:02<00:00,  3.99it/s]


,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),樣本勝率(%),周勝率(%),月勝率(%),年勝率(%),avg_win/avg_loss,profit_factor
0% ~ 10%,-0.39,-3.84,-0.27,-86.85,-201.90,42.69,32.73,14.06,0.00,0.71,0.53
10% ~ 20%,-0.18,-2.24,-0.19,-59.54,-88.72,47.19,41.01,23.44,0.00,0.77,0.69
20% ~ 30%,-0.19,-1.96,-0.18,-55.56,-78.20,46.09,35.97,21.88,0.00,0.81,0.70
30% ~ 40%,-0.12,-2.36,-0.17,-45.75,-60.64,46.66,38.13,28.12,0.00,0.77,0.67
40% ~ 50%,-0.03,-1.03,-0.14,-16.14,-17.31,48.60,43.88,39.06,33.33,0.89,0.84
50% ~ 60%,-0.01,-0.51,-0.12,-7.15,-7.23,47.67,45.32,37.50,33.33,1.01,0.92
60% ~ 70%,0.07,2.22,1.98,-2.24,-1.79,54.74,61.87,73.44,100.00,1.20,1.45
70% ~ 80%,0.11,2.75,3.26,-2.25,-2.18,54.28,62.23,75.00,100.00,1.34,1.59
80% ~ 90%,0.13,2.61,2.82,-3.21,-2.56,53.27,58.99,73.44,100.00,1.38,1.57
90% ~ 100%,0.25,3.64,5.09,-3.49,-2.36,56.14,65.83,78.12,100.00,1.51,1.93


In [231]:
Top_n =10 #選股數量

In [232]:
from datetime import datetime
import tejapi
benchmark = tejapi.get('TWN/APIPRCD', coid='IX0001',opts={'columns': ['mdate','close_d']}, mdate={'gte': '2010-01-01', 'lte':'2025-04-25'}, paginate=True)
benchmark["mdate"] = pd.to_datetime(benchmark["mdate"])
benchmark["mdate"] = benchmark["mdate"].dt.strftime("%Y-%m-%d")
benchmark_return = benchmark['close_d']
benchmark_return.index = CacheHandler['Adj_Close'].index
bench_exp_ret = benchmark_return.shift(-1)

In [261]:
def calculate_return_series(ic_ir_factor, Top_n, exp_ret, close_px, plot=False, stop_loss_threshold=-0.03):
    Top_n = int(Top_n)
    stop_loss_records = []
    month_end_dates = ic_ir_factor.groupby(ic_ir_factor.index.to_period("W")).apply(lambda x: x.index.max()).values
    latest_signal_date = month_end_dates[-1]
    print(f"🕒 最新一期更改進場持股的日期是：{latest_signal_date}")
    def fill_signal(signal, weight):
        filled_signal = pd.DataFrame(index=weight.index, columns=weight.columns)
        first_date = month_end_dates[0]
        filled_signal.loc[:first_date] = signal.loc[first_date].values
        for i in range(len(month_end_dates) - 1):
            current_date, next_date = month_end_dates[i], month_end_dates[i + 1]
            date_range = weight.loc[current_date:next_date].index
            filled_signal.loc[date_range] = signal.loc[current_date].values
        return filled_signal.dropna(how='all') > 0

    # Step 1: 產生進場信號
    signal_bool = {
        "ic_ir": (ic_ir_factor.rank(axis=1, ascending=False, method='first') <= Top_n).astype(int),
    }

    filled_signals = {
        k: fill_signal(v, ic_ir_factor if k == "ic_ir" else None)
        for k, v in signal_bool.items()
    }

    W_df = {k: signal.astype(int) / Top_n for k, signal in filled_signals.items()}

    def calculate_return(weighting):
        delta_weight = weighting.diff()
        buy_fee = delta_weight[delta_weight > 0] * (0.001425 * 0.3 + 0.003)
        sell_fee = delta_weight[delta_weight < 0].abs() * (0.001425 * 0.3)

        effective_weight = weighting.copy()
        # 新增一個字典來記錄每個股票的停損日期
        stop_loss_dates = {}

        for i in range(len(month_end_dates) - 1):
            signal_date = month_end_dates[i]
            if signal_date not in weighting.index:
                continue

            idx = weighting.index.get_loc(signal_date)
            if idx + 2 >= len(weighting):  # 需 entry、exit
                continue

            entry_date = weighting.index[idx + 1]
            exit_date = weighting.index[idx + 2]
            active_stocks = weighting.columns[weighting.loc[signal_date] > 0]

            dropped = []
            for stock in active_stocks:
                # 如果股票已經被停損，跳過
                if stock in stop_loss_dates and stop_loss_dates[stock] <= signal_date:
                    continue

                if (
                    entry_date not in exp_ret.index
                    or exit_date not in exp_ret.index
                    or stock not in exp_ret.columns
                ):
                    continue

                # 累積報酬
                ret_path = exp_ret[stock].loc[entry_date:exit_date].fillna(0)
                cum_ret = ret_path.cumsum()  # 改用累積報酬

                # 若觸發停損
                if (cum_ret < stop_loss_threshold).any():
                    if stock in close_px.columns and entry_date in close_px.index and exit_date in close_px.index:
                        # 找到exit_date的前一個交易日
                        prev_close = close_px.at[exit_date, stock]
                        exit_close = close_px.at[exit_date, stock]
                        limit_threshold = prev_close * 0.91

                        if exit_close <= limit_threshold:
                            # 延後一天出場
                            next_idx = weighting.index.get_loc(exit_date) + 1
                            if next_idx < len(weighting.index):
                                delay_exit = weighting.index[next_idx]
                                dropped.append(stock)
                                stop_loss_records.append((delay_exit, stock))
                                stop_loss_dates[stock] = delay_exit  # 記錄停損日期
                                # 將停損股票的權重設為0，直到下次調倉日
                                next_signal_date = month_end_dates[i + 1] if i + 1 < len(month_end_dates) else weighting.index[-1]
                                effective_weight.loc[delay_exit:next_signal_date, stock] = 0
                            continue

                    # 否則照當天收盤出場
                    dropped.append(stock)
                    stop_loss_records.append((exit_date, stock))
                    stop_loss_dates[stock] = exit_date  # 記錄停損日期
                    # 將停損股票的權重設為0，直到下次調倉日
                    next_signal_date = month_end_dates[i + 1] if i + 1 < len(month_end_dates) else weighting.index[-1]
                    effective_weight.loc[exit_date:next_signal_date, stock] = 0

        daily_ret_matrix = effective_weight * exp_ret
        daily_ret = daily_ret_matrix.sum(axis=1)
        cost = buy_fee.fillna(0).sum(axis=1) + sell_fee.fillna(0).sum(axis=1)
        return daily_ret - cost, daily_ret_matrix, effective_weight

    weightings = {k: W_df[k] for k in W_df}
    return_series = {}
    ret_matrix_dict = {}
    effective_weight_dict = {}

    for k, w in weightings.items():
        r, r_matrix, e_weight = calculate_return(w)
        return_series[k] = r
        ret_matrix_dict[k] = r_matrix
        effective_weight_dict[k] = e_weight
        # 更新weightings為停損後的權重
        weightings[k] = e_weight

    if plot:
        for k, v in return_series.items():
            v.iplot(title=k)

    bt_ret = pd.concat({
        'ic_ir_return': return_series['ic_ir'],
    }, axis=1).loc['2020':]

    return bt_ret, weightings, stop_loss_records, ret_matrix_dict, effective_weight_dict

bt_ret, weightings, stop_loss_records, ret_matrix_dict, effective_weight_dict = calculate_return_series(
    ic_ir_factor=factor[mask].loc['2020':],
    Top_n=10,
    exp_ret=exp_ret_fu.loc['2020':],
    close_px=low_matrix.loc['2020':],  
    plot=False
)


🕒 最新一期更改進場持股的日期是：2025-04-25T00:00:00.000000000


In [289]:
signal_dates = effective_weight_dict['ic_ir'].index.to_series().groupby(lambda x: x.to_period("W")).apply(lambda x: x.max()).values

# 最新一期與前一期
latest_date = signal_dates[-1]
previous_date = signal_dates[-2]

In [290]:
# 取出進場信號（權重 > 0）的股票
latest_stocks = effective_weight_dict['ic_ir'].loc[latest_date]
latest_stocks = latest_stocks[latest_stocks > 0].index

previous_stocks = effective_weight_dict['ic_ir'].loc[previous_date]
previous_stocks = previous_stocks[previous_stocks > 0].index


In [291]:
latest_set = set(latest_stocks)
previous_set = set(previous_stocks)

added = latest_set - previous_set       # 新增
removed = previous_set - latest_set     # 被剔除
unchanged = latest_set & previous_set   # 持續持有

print(f"前一期調倉日：{previous_date}")
print(f"最新一期調倉日：{latest_date}")
print("\n新增股票：", sorted(added))
print("剔除股票：", sorted(removed))
print("持續持有：", sorted(unchanged))


前一期調倉日：2025-04-18T00:00:00.000000000
最新一期調倉日：2025-04-25T00:00:00.000000000

新增股票： []
剔除股票： []
持續持有： ['2049', '2317', '2345', '3189', '3533', '3706', '3711', '6488', '8046', '8299']


In [282]:
stop_loss_df = pd.DataFrame(stop_loss_records, columns=['exit_date', 'stock'])
stop_loss_df = stop_loss_df.sort_values('exit_date')
stop_loss_df.tail(20)  

,exit_date,stock
81,2024-03-12,3653
82,2024-03-12,3702
83,2024-04-02,2439
84,2024-05-07,2548
85,2024-05-07,5534
86,2024-05-28,9904
87,2024-06-04,2392
88,2024-07-23,3081
89,2024-08-13,2049
90,2024-08-13,5269


In [266]:
import warnings
warnings.filterwarnings("ignore")
def max_drawdown(prices):
    # 計算累計的最大值
    cumulative_max = prices.cummax()
    # 計算回撤 (Drawdown)
    drawdown = (prices - cumulative_max) / cumulative_max
    # 計算最大回撤 (MDD)
    mdd = drawdown.min()
    return mdd
display(pd.concat({"CAGR(%)":bt_ret.mean() * 252 * 100,
        'Sharpe' : bt_ret.mean() / bt_ret.std()*252**0.5,
        'Calmar':bt_ret.cagr() / abs(bt_ret.max_drawdown()),
        'MDD(%)' : bt_ret.max_drawdown()*100,
        '單利MDD(%)' : max_drawdown(bt_ret.cumsum().add(1))*100,
        '樣本勝率(%)' : bt_ret.apply(lambda X:((X.dropna()>0).sum() / X.dropna().shape[0])*100),
        '周勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('W').prod().sub(1)>0).sum() / X.dropna().add(1).resample('W').prod().sub(1).dropna().shape[0])*100),
        '月勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('ME').prod().sub(1)>0).sum() / X.dropna().add(1).resample('ME').prod().sub(1).shape[0])*100),
        '年勝率(%)' : bt_ret.apply(lambda X:((X.dropna().add(1).resample('YE').prod().sub(1)>0).sum() / X.dropna().add(1).resample('YE').prod().sub(1).shape[0])*100),
        "avg_win(%)": bt_ret.apply(lambda X: X[X > 0].mean() * 100),
        "avg_loss(%)": bt_ret.apply(lambda X: X[X < 0].mean() * 100),
        'avg(%)':bt_ret.apply(lambda X:X.mean() * 100),
        'avg_win/avg_loss' : bt_ret.apply(lambda X:(X[X > 0].mean() / abs(X[X < 0].mean()))),
        '平均交易報酬(%)': bt_ret.apply(lambda x: (x[x != 0].mean()) * 100),
        'profit_factor' : bt_ret.profit_factor(),
        },axis = 1).round(2))
bt_ret.cumsum().ffill().iplot(title='Equity Curve')
bt_ret.to_drawdown_series().iplot(title = 'Drawdown')
pd.concat({
    f'equal({round(weightings["ic_ir"].loc["2020":].sum(axis=1).mean(), 4)})': weightings['ic_ir'].loc["2020":].sum(axis=1),
}, axis=1).iplot(title='現金使用率')


,CAGR(%),Sharpe,Calmar,MDD(%),單利MDD(%),樣本勝率(%),周勝率(%),月勝率(%),年勝率(%),avg_win(%),avg_loss(%),avg(%),avg_win/avg_loss,平均交易報酬(%),profit_factor
ic_ir_return,39.26,1.72,1.08,-25.57,-28.78,55.98,57.91,68.75,83.33,1.07,-1.01,0.16,1.06,0.16,1.35


In [293]:
bt_ret_last10days

,ic_ir_return
mdate,
2025-04-21,0.000000
2025-04-22,-1.056978
2025-04-23,5.876898
2025-04-24,0.292680
2025-04-25,3.709176


In [268]:
bt_ret_last10days = bt_ret.shift(2).iloc[-5:] * 100
bt_ret_last10days.iloc[0] = 0
bt_ret_last10days.cumsum().iplot(title='累積報酬 % ')

In [269]:
績效回顧 = bt_ret.reindex(exp_ret.index).shift(1)
display((績效回顧.iloc[-21:]*100).round(2))

,ic_ir_return
mdate,
2025-03-26,0.10
2025-03-27,-1.94
2025-03-28,-2.82
2025-03-31,1.78
2025-04-01,0.63
2025-04-02,-7.98
2025-04-07,-2.98
2025-04-08,-8.27
2025-04-09,9.79


In [270]:
def check_roll_contracts_for_weekly_strategy(weight_df, top_contracts, target_date="2023-03-31"):
    import pandas as pd

    # 轉換成 datetime
    target_date = pd.to_datetime(target_date)

    # 找到該週的最後一個交易日
    week_dates = weight_df.index.to_series().dt.to_period("W")
    week_last_day = weight_df[week_dates == target_date.to_period("W")].index.max()

    if pd.isna(week_last_day):
        print(f"找不到 {target_date.strftime('%Y-%m-%d')} 所在週的最後交易日")
        return

    # 找下一個交易日作為進場日
    all_dates = weight_df.index.sort_values()
    next_day_arr = all_dates[all_dates > week_last_day]
    if len(next_day_arr) == 0:
        print(f"{week_last_day.date()} 之後沒有交易日")
        return

    signal_date = week_last_day
    next_day = next_day_arr[0]

    print(f"\n週末訊號日：{signal_date.date()}，進場日：{next_day.date()}")

    # 當週持股
    held = weight_df.loc[signal_date]
    held = held[held > 0]

    # 查期貨合約
    for coid in held.index:
        match = top_contracts[
            (top_contracts['年月日'] == next_day) &
            (top_contracts['標的證券'].astype(str).str.contains(str(coid)))
        ]
        if match.empty:
            print(f"{coid}：{next_day.date()} 無期貨資料")
        else:
            contract = match.iloc[0]['證券代碼']
            print(f"{coid} → 合約：{contract}")

In [271]:
check_roll_contracts_for_weekly_strategy(effective_weight_dict['ic_ir'], top_contracts, target_date="2025-04-19")


週末訊號日：2025-04-18，進場日：2025-04-21
2049 → 合約：QMF202505 小型上銀期 2025/05
2317 → 合約：DHF202505 鴻海期貨 2025/05
2345 → 合約：SEF202505 小型智邦期貨 2025/05
3189 → 合約：IXF202505 景碩期貨 2025/05
3533 → 合約：RFF202505 小型嘉澤期貨 2025/05
3706 → 合約：PSF202505 神達期貨 2025/05
3711 → 合約：OZF202505 日月光期貨 2025/05
6488 → 合約：PBF202505 小型環球晶 2025/05
8046 → 合約：QSF202505 小型南電期 2025/05
8299 → 合約：QNF202505 小型群聯期 2025/05


In [272]:
def get_contracts_from_weights(weight_df, top_contracts, target_date=None):
    import pandas as pd

    # 確保 top_contracts 中日期格式正確
    top_contracts = top_contracts.copy()
    top_contracts['年月日'] = pd.to_datetime(top_contracts['年月日'])

    result = []

    # 若有指定日期，轉成 list；否則用全部日期
    if target_date is not None:
        dates_to_check = [pd.to_datetime(target_date)]
    else:
        dates_to_check = weight_df.index

    for dt in dates_to_check:
        if dt not in weight_df.index:
            print(f"❌ {dt.date()} 不在 weight_df 中")
            continue

        held = weight_df.loc[dt]
        held = held[held > 0]

        if held.empty:
            print(f"⚠️ {dt.date()} 無持倉")
            continue

        for coid, wt in held.items():
            match = top_contracts[
                (top_contracts['年月日'] == dt) &
                (top_contracts['標的證券'].astype(str).str.contains(str(coid)))
            ]
            if match.empty:
                contract = None
            else:
                contract = match.iloc[0]['證券代碼']
            result.append({
                "日期": dt,
                "股票代碼": coid,
                "權重": wt,
                "期貨合約": contract
            })

    return pd.DataFrame(result)


In [292]:
get_contracts_from_weights(effective_weight_dict['ic_ir'], top_contracts, target_date="2025-04-25")

,日期,股票代碼,權重,期貨合約
0,2025-04-25,2049,0.1,QMF202505 小型上銀期 2025/05
1,2025-04-25,2317,0.1,DHF202505 鴻海期貨 2025/05
2,2025-04-25,2345,0.1,SEF202505 小型智邦期貨 2025/05
3,2025-04-25,3189,0.1,IXF202505 景碩期貨 2025/05
4,2025-04-25,3533,0.1,RFF202505 小型嘉澤期貨 2025/05
5,2025-04-25,3706,0.1,PSF202505 神達期貨 2025/05
6,2025-04-25,3711,0.1,OZF202505 日月光期貨 2025/05
7,2025-04-25,6488,0.1,PBF202505 小型環球晶 2025/05
8,2025-04-25,8046,0.1,LYF202505 南電期貨 2025/05
9,2025-04-25,8299,0.1,QNF202505 小型群聯期 2025/05
